In [25]:
import pandas as pd

df = pd.read_excel("sp500_data.xlsx")

df = df[['stock_symbol', 'date', 'open']]

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

df.head()

,stock_symbol,date,open
0,MMM,2024-03-18,85.453221
1855,ANSS,2024-03-18,330.660004
15527,LOW,2024-03-18,241.268927
15474,L,2024-03-18,76.730276
15421,LMT,2024-03-18,424.682620


In [26]:
# Prepare data
df_sorted = df.sort_values('date')
dates = df_sorted['date'].unique()
periods = {date: idx+1 for idx, date in enumerate(dates)}
df_sorted['period'] = df_sorted['date'].map(periods)
T = len(dates)

# Filter stocks with complete history
valid_stocks = []
for stock in df_sorted['stock_symbol'].unique():
    if len(df_sorted[df_sorted['stock_symbol'] == stock]['period'].unique()) == T:
        valid_stocks.append(stock)

# Filter dataframe to only valid stocks
df_filtered = df_sorted[df_sorted['stock_symbol'].isin(valid_stocks)]

df = df_filtered.copy()

In [30]:
# risk_adjusted_sp500.py
# ------------------------------------------------------------
# Portfolio optimiser: maximise   Σ w_i (μ_i - λ σ_i)
# No clairvoyance – all inputs at week t use data ≤ t‑1.
# ------------------------------------------------------------

import pandas as pd
import numpy as np
from pathlib import Path
from pulp import (
    LpProblem, LpVariable, LpMaximize, lpSum, value, LpStatus, PULP_CBC_CMD
)

# ------------------------- configuration --------------------
DATA_FILE      = Path("sp500_data.xlsx")  # weekly OHLCV data
LOOKBACK_WEEKS = 12                       # trailing window for μ, σ
LAMBDA         = 1.0                      # risk‑aversion coefficient λ
MAX_WEIGHT     = 0.05                     # per‑stock cap (None → no cap)
INITIAL_CASH   = 10_000                  # starting capital for back‑test
# ------------------------------------------------------------


def load_weekly_prices(path: Path) -> pd.DataFrame:
    """Read the Excel sheet and keep only the fields we need."""
    df = pd.read_excel(path, sheet_name=0, parse_dates=["date"])
    # Prepare data
    df_sorted = df.sort_values('date')
    dates = df_sorted['date'].unique()
    periods = {date: idx+1 for idx, date in enumerate(dates)}
    df_sorted['period'] = df_sorted['date'].map(periods)
    T = len(dates)
    
    # Filter stocks with complete history
    valid_stocks = []
    for stock in df_sorted['stock_symbol'].unique():
        if len(df_sorted[df_sorted['stock_symbol'] == stock]['period'].unique()) == T:
            valid_stocks.append(stock)
    
    # Filter dataframe to only valid stocks
    df_filtered = df_sorted[df_sorted['stock_symbol'].isin(valid_stocks)]

    return df_filtered[["stock_symbol", "date", "close"]]


def add_weekly_returns(prices: pd.DataFrame) -> pd.DataFrame:
    """Append simple weekly returns by ticker."""
    prices["ret"] = (
        prices.groupby("stock_symbol")["close"]
        .pct_change()
        .fillna(0.0)
    )
    return prices


def trailing_stats(history: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Compute μ (mean return) and σ (volatility) over the last `window` weeks
    for each stock, based solely on the slice supplied in `history`.
    """
    recent = (
        history.groupby("stock_symbol")
        .tail(window)
        .groupby("stock_symbol")["ret"]
    )
    mu     = recent.mean()
    sigma  = recent.std(ddof=0)
    stats  = pd.DataFrame({"mu": mu, "sigma": sigma})
    stats["score"] = stats["mu"] - LAMBDA * stats["sigma"]
    stats.dropna(inplace=True)
    return stats


def optimise_portfolio(scores: pd.Series) -> pd.Series:
    """
    Solve a tiny LP: maximise Σ w_i score_i,  s.t. Σ w_i = 1,
    0 ≤ w_i ≤ MAX_WEIGHT (if set).
    Returns a weight vector indexed by stock_symbol.
    """
    tickers = scores.index.tolist()
    prob    = LpProblem("risk_adjusted_return", LpMaximize)

    w = {
        t: LpVariable(f"w_{t}", lowBound=0.0, upBound=MAX_WEIGHT)
        for t in tickers
    }
    prob += lpSum(w[t] * scores[t] for t in tickers)          # objective
    prob += lpSum(w.values()) == 1.0                          # fully invested

    prob.solve(PULP_CBC_CMD(msg=False))
    if LpStatus[prob.status] != "Optimal":
        raise RuntimeError("LP did not converge")

    return pd.Series({t: value(w[t]) for t in tickers})


def backtest(prices: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    weeks = sorted(prices["date"].unique())
    wealth = INITIAL_CASH
    portfolio_value = {}
    wealth_log = []
    portfolio_log = []
    transactions_log = []

    prev_weights = pd.Series(dtype=float)

    for i in range(LOOKBACK_WEEKS + 1, len(weeks)):
        week = weeks[i]
        past_data = prices[prices["date"] < week]
        stats = trailing_stats(past_data, LOOKBACK_WEEKS)
        weights = optimise_portfolio(stats["score"])

        prev_week = prices[prices["date"] == weeks[i - 1]]
        curr_week = prices[prices["date"] == week]
        merged = (
            prev_week.merge(curr_week, on="stock_symbol", suffixes=("_prev", "_curr"))
            .set_index("stock_symbol")
            .loc[weights.index]
        )

        realised_ret = ((merged["close_curr"] / merged["close_prev"]) - 1.0).fillna(0)
        portfolio_ret = (weights * realised_ret).sum()

        wealth *= 1 + portfolio_ret
        wealth_log.append({"date": weeks[i], "wealth": wealth})

        if prev_weights.empty:
            # First week with investment — all are buys
            for stock, weight in weights.items():
                amount = round(weight * wealth, 2)
                if amount > 0:
                    transactions_log.append({
                        "date": weeks[i],
                        "stock_symbol": stock,
                        "action": "buy",
                        "amount": amount
                    })
        else:
            # Compute buy/sell deltas relative to last week
            prev_alloc = (prev_weights * wealth / (1 + portfolio_ret)).reindex(weights.index).fillna(0)
            curr_alloc = weights * wealth
            delta = curr_alloc - prev_alloc
            for stock, change in delta.items():
                if abs(change) > 1e-6:
                    transactions_log.append({
                        "date": weeks[i],
                        "stock_symbol": stock,
                        "action": "buy" if change > 0 else "sell",
                        "amount": round(abs(change), 2)
                    })

        prev_weights = weights
        wealth *= (1 + portfolio_ret)
        portfolio_value[week] = wealth

        # Store portfolio composition
        for stock, w in weights.items():
            portfolio_log.append({
                "date": week,
                "stock_symbol": stock,
                "weight": round(w, 4),
                "value_usd": round(w * wealth, 2)
            })

    summary = pd.DataFrame([{"date": d, "wealth": v} for d, v in portfolio_value.items()])
    return summary, pd.DataFrame(portfolio_log), pd.DataFrame(transactions_log), pd.DataFrame(wealth_log)

if __name__ == "__main__":
    prices = add_weekly_returns(load_weekly_prices(DATA_FILE))

    # Latest weights
    latest_window = prices["date"].max()
    stats_now = trailing_stats(prices[prices["date"] < latest_window], LOOKBACK_WEEKS)
    weights = optimise_portfolio(stats_now["score"])
    print("Optimal weights (latest):")
    print(weights[weights > 0].sort_values(ascending=False))

    # Back-test with detailed logging
    summary_df, portfolio_df, tx_df, wealth_log = backtest(prices)
    
    print("\nBack-test summary:")
    print(summary_df.tail())

    print("\nSample portfolio allocations:")
    print(portfolio_df.head())

    print("\nSample transactions:")
    print(tx_df.head())

    print("\nWealth log:")
    print(wealth_log.head())

Optimal weights (latest):
AEP     0.05
AMGN    0.05
BRO     0.05
CI      0.05
CME     0.05
COR     0.05
ELV     0.05
EXC     0.05
GL      0.05
JNJ     0.05
K       0.05
MCK     0.05
PGR     0.05
ROL     0.05
RSG     0.05
SO      0.05
VICI    0.05
VRSN    0.05
VRTX    0.05
WEC     0.05
dtype: float64

Back-test summary:
         date        wealth
35 2025-02-17  12338.509289
36 2025-02-24  12576.487916
37 2025-03-03  12376.032825
38 2025-03-10  12013.159466
39 2025-03-17  12325.566667

Sample portfolio allocations:
        date stock_symbol  weight  value_usd
0 2024-06-17            A     0.0        0.0
1 2024-06-17         AAPL     0.0        0.0
2 2024-06-17         ABBV     0.0        0.0
3 2024-06-17         ABNB     0.0        0.0
4 2024-06-17          ABT     0.0        0.0

Sample transactions:
        date stock_symbol action  amount
0 2024-06-17          APD    buy  501.42
1 2024-06-17          AVB    buy  501.42
2 2024-06-17          CHD    buy  501.42
3 2024-06-17          CN

In [28]:
# Save portfolio and transaction logs to CSVs
portfolio_df.to_csv("portfolio_log.csv", index=False)
tx_df.to_csv("transactions_log.csv", index=False)
